In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install torch transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import Dataset
import pandas as pd

# Load your cleaned dataset
df = pd.read_csv('/content/drive/MyDrive/CNN_Dataset_Preprocessing/final_cleaned_dataset.csv')

# Create a Hugging Face Dataset
data = Dataset.from_pandas(df[['article_cleaned', 'summary_cleaned']])


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Assuming `data` is your Hugging Face dataset
train_data, eval_data = train_test_split(df[['article_cleaned', 'summary_cleaned']], test_size=0.2, random_state=42)

# Convert them to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments

# Load the tokenizer and model
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(batch):
    # Tokenize the articles
    inputs = tokenizer(
        batch['article_cleaned'],
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

    # Tokenize the summaries (these will be the labels)
    labels = tokenizer(
        batch['summary_cleaned'],
        padding='max_length',
        truncation=True,
        max_length=128,  # Adjust max_length based on your summary length
        return_tensors='pt'
    )

    # Add labels to the inputs dictionary
    inputs['labels'] = labels['input_ids']
    return inputs


In [ ]:
# Tokenize the datasets
tokenized_train_data = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_data = eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# Set the format of datasets for training
tokenized_train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_eval_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=228211767d33e29eb9c6529f53d507a3bce2680b87ecd18ad4af3fa6d9e8e588
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


##First training

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    save_total_limit=2,
    remove_unused_columns=False,
    logging_dir='./logs',
    logging_steps=100,
    report_to='none'
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_eval_data,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-617fc23d66ac>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.186900,1.103268
2,0.771100,1.091591
3,0.467400,1.152237


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1200, training_loss=0.8999755255381267, metrics={'train_runtime': 982.0958, 'train_samples_per_second': 2.444, 'train_steps_per_second': 1.222, 'total_flos': 2600525522534400.0, 'train_loss': 0.8999755255381267, 'epoch': 3.0})

##Second Training

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate

# Initialize ROUGE for evaluation
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Filter out any None values
    predictions = [pred for pred in predictions if pred is not None]
    labels = [label for label in labels if label is not None]

    # Decode predictions and labels to strings
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute metrics like BLEU or ROUGE
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    return result


In [ ]:
# Create a data collator with padding and dynamic batch sizes
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# Define training arguments with early stopping
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    save_total_limit=2,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    logging_dir='./logs',
    logging_steps=100,
    eval_accumulation_steps=10,
    report_to='none',
    predict_with_generate=True,  # Generate summaries during evaluation
    generation_max_length=150,
    generation_num_beams=4
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer, EarlyStoppingCallback

# Define early stopping callback with your desired patience
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Number of evaluations with no improvement before stopping
    early_stopping_threshold=0.01  # Minimum change to qualify as an improvement
)

# Initialize the Trainer with the early stopping callback
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_eval_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]  # Set the early stopping callback
)


<ipython-input-15-f9777dece711>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.225200,1.097169,0.352740,0.156076,0.255102,0.254518
2,0.818400,1.117260,0.347831,0.149838,0.252792,0.252539
3,0.556200,1.173522,0.363844,0.150683,0.254632,0.253925
4,0.320900,1.343420,0.366476,0.159018,0.260154,0.260038


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=800, training_loss=0.8057809400558472, metrics={'train_runtime': 1476.4023, 'train_samples_per_second': 5.419, 'train_steps_per_second': 1.355, 'total_flos': 3467367363379200.0, 'train_loss': 0.8057809400558472, 'epoch': 4.0})

In [ ]:
test_article = """Artificial Intelligence (AI) is rapidly transforming various industries, from healthcare to finance. One of the most promising applications of AI is in natural language processing (NLP), which enables machines to understand and generate human language. A key component of NLP is text summarization, a technique that condenses long pieces of text into shorter versions while preserving the most important information.

Traditional text summarization techniques often rely on statistical methods or rule-based approaches. However, recent advancements in deep learning have led to the development of more sophisticated models that can generate more accurate and coherent summaries. These models, such as Transformers, are trained on massive datasets of text and can learn complex patterns in language.

Text summarization has numerous practical applications. For example, it can be used to generate news summaries, create abstracts for research papers, and provide concise summaries of lengthy documents. Additionally, it can be used to improve information retrieval by helping users quickly identify relevant information.

As AI continues to evolve, we can expect to see further improvements in text summarization techniques. This will enable us to better understand and process information, making it easier to stay informed and make informed decisions."""

inputs = tokenizer(test_article, return_tensors="pt", truncation=True, max_length=512).to(model.device)
summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Generated Summary:", summary)

Generated Summary: glossy text summarization technique condenses long pieces of text into shorter versions while preserving the most important information. It can be used to generate news summaries, create abstracts for research papers, and provide concise summaries of lengthy documents. As AI continues to evolve, we can expect to see further improvements in text summarizing techniques


In [ ]:
# Save the model and tokenizer
output_dir = './saved_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")


Model and tokenizer saved to ./saved_model


In [ ]:
output_dir = '/content/drive/MyDrive/CNN_Dataset_Preprocessing/saved_model'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir} in Google Drive")


Model and tokenizer saved to /content/drive/MyDrive/CNN_Dataset_Preprocessing/saved_model in Google Drive


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the model and tokenizer
output_dir = '/content/drive/MyDrive/CNN_Dataset_Preprocessing/saved_model'
loaded_model = BartForConditionalGeneration.from_pretrained(output_dir)
loaded_tokenizer = BartTokenizer.from_pretrained(output_dir)

print("Model and tokenizer successfully loaded!")


/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


Model and tokenizer successfully loaded!


In [ ]:
import transformers
import torch

print("Transformers version:", transformers.__version__)
print("Torch version:", torch.__version__)


Transformers version: 4.46.2
Torch version: 2.5.1+cu121


##Another training

In [ ]:
from transformers import DataCollatorForSeq2Seq

# Create a data collator with dynamic padding
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True  # Dynamic padding
)


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',                # Evaluate at the end of each epoch
    per_device_train_batch_size=4,              # Batch size per device
    per_device_eval_batch_size=4,               # Evaluation batch size
    num_train_epochs=10,                        # Total number of epochs
    save_total_limit=2,                         # Limit the number of checkpoints saved
    save_strategy='epoch',                      # Save at the end of each epoch
    load_best_model_at_end=True,                # Load best model at the end of training
    metric_for_best_model='eval_loss',          # Use loss to determine best model
    greater_is_better=False,                    # Lower loss is better
    logging_dir='./logs',                       # Directory for logs
    logging_steps=100,                          # Log every 100 steps
    eval_accumulation_steps=10,                 # Evaluate every 10 steps
    report_to="none",                           # No external reporting
    predict_with_generate=True,                 # Generate summaries during evaluation
    generation_num_beams=4,                     # Beam search for better summaries
    label_smoothing_factor=0.1,                 # Smooth labels to improve generalization
    gradient_accumulation_steps=4,              # Accumulate gradients to simulate larger batch size
    lr_scheduler_type='cosine',                 # Cosine decay for learning rate
    warmup_steps=500,                           # Warm-up phase for the first 10% of steps
    fp16=True,                                  # Mixed-precision training for faster performance
    adafactor=True                              # Use Adafactor optimizer (memory-efficient)
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1725: FutureWarning: `--adafactor` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--optim adafactor` instead
  warnings.warn(


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,              # Larger batch size
    per_device_eval_batch_size=8,
    num_train_epochs=15,                        # Train for more epochs
    save_total_limit=2,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    logging_dir='./logs',
    logging_steps=50,                           # Log frequently
    eval_accumulation_steps=10,
    report_to="none",
    predict_with_generate=True,
    generation_num_beams=6,                     # More diverse beams
    label_smoothing_factor=0.1,
    gradient_accumulation_steps=2,              # Simulate larger batches
    lr_scheduler_type='cosine_with_restarts',   # Better learning rate decay
    warmup_ratio=0.1,
    fp16=True,                                  # Mixed precision training
    learning_rate=3e-5,                         # Lower LR
    adafactor=True                              # Efficient optimizer
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1725: FutureWarning: `--adafactor` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--optim adafactor` instead
  warnings.warn(


In [ ]:
training_args.gradient_accumulation_steps = 4  # Accumulate gradients across 4 steps


In [ ]:
import evaluate

# Load evaluation metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = [pred for pred in predictions if pred is not None]
    labels = [label for label in labels if label is not None]

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE metrics
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    # BLEU metrics
    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)

    # METEOR metrics
    meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)

    # Combine all results
    result = {
        "rouge1": rouge_result["rouge1"],
        "rouge2": rouge_result["rouge2"],
        "rougeL": rouge_result["rougeL"],
        "rougeLsum": rouge_result["rougeLsum"],
        "bleu": bleu_result["bleu"],
        "meteor": meteor_result["meteor"]
    }
    return result


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=2,  # Stop training after 2 epochs with no improvement
    early_stopping_threshold=0.001  # Require at least 0.1% improvement
)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_eval_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)


<ipython-input-21-a8bfc4f32fb5>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()


AttributeError: `AcceleratorState` object has no attribute `distributed_type`. This happens if `AcceleratorState._reset_state()` was called and an `Accelerator` or `PartialState` was not reinitialized.